# Collocations

In [ ]:
#load all libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
import string

Data: https://www.kaggle.com/datafiniti/hotel-reviews/data

In [ ]:
#load reviews data
reviews = pd.read_csv('cleanaarogya.csv')

In [ ]:
reviews.head(2)

,REVIEW
0,cowinapp not ios aarogyasetu user register moh...
1,corona warn app germany permanently show risk ...


Extract only the reviews...

In [ ]:
comments = reviews['REVIEW']

## Preprocessing

In [ ]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [ ]:
comments = comments.astype('str')

In [ ]:
#remove non-ascii characters
comments = comments.map(lambda x: _removeNonAscii(x))

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#get stop words of all languages
STOPWORDS_DICT = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}

In [ ]:
#function to detect language based on # of stop words for particular language
def get_language(text):
    words = set(nltk.wordpunct_tokenize(text.lower()))
    lang = max(((lang, len(words & stopwords)) for lang, stopwords in STOPWORDS_DICT.items()), key = lambda x: x[1])[0]
    if lang == 'english':
        return True
    else:
        return False

In [ ]:
#filter for only english comments
eng_comments=comments[comments.apply(get_language)]

In [ ]:
eng_comments.head()

0     cowinapp not ios aarogyasetu user register moh...
10    swarraj nadjanadika sunetrac registration aaro...
20    covidvaccine registration open senior citizen ...
21        sivismyname not function try aarogya setu app
23    rupagulab successfully small pox vaccine polio...
Name: REVIEW, dtype: object

In [ ]:
#drop duplicates
eng_comments.drop_duplicates(inplace=True)

In [ ]:
#load spacy
nlp = spacy.load('en')

In [ ]:
#function to clean and lemmatize comments
def clean_comments(text):
    #remove punctuations
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    #use spacy to lemmatize comments
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [ ]:
#apply function to clean and lemmatize comments
lemmatized = eng_comments.map(clean_comments)

In [ ]:
12#make sure to lowercase everything
lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])

In [ ]:
lemmatized.head()

0     [cowinapp, not, ios, aarogyasetu, user, regist...
10    [swarraj, nadjanadika, sunetrac, registration,...
20    [covidvaccine, registration, open, senior, cit...
21    [sivismyname, not, function, try, aarogya, set...
23    [rupagulab, successfully, small, pox, vaccine,...
Name: REVIEW, dtype: object

In [ ]:
#turn all comments' tokens into one single list
unlist_comments = [item for items in lemmatized for item in items]

## Initialize NLTK's Bigrams/Trigrams Finder

In [ ]:
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_comments)

## 1. Counting Frequencies of Adjacent Words
- Main idea: simply order by frequency
- Issues: too sensitive to very frequent pairs and pronouns/articles/prepositions come up often
- Solution: filter for only adjectives and nouns

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

,bigram,freq
0,"(aarogya, setu)",10069
1,"(setu, app)",5673
2,"(app, not)",1368
3,"(aarogyasetu, app)",1028
4,"(download, aarogya)",959


In [ ]:
bigramFreqTable[:10]

,bigram,freq
12,"(aarogya, setu)",10069
48,"(setu, app)",5673
64,"(app, not)",1368
70,"(aarogyasetu, app)",1028
574,"(download, aarogya)",959
13,"(setu, not)",734
2994,"(use, aarogya)",400
6,"(mohfw, india)",390
925,"(not, work)",371
540,"(not, aarogya)",367


In [ ]:
#get english stopwords
en_stopwords = set(stopwords.words('english'))

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

,bigram,freq
12,"(aarogya, setu)",10069
48,"(setu, app)",5673
70,"(aarogyasetu, app)",1028
574,"(download, aarogya)",959
2994,"(use, aarogya)",400
1291,"(app, httpst)",340
1600,"(contact, trace)",326
439,"(self, declaration)",298
3300,"(smart, phone)",291
440,"(declaration, form)",288


In [ ]:
trigram_freq = trigramFinder.ngram_fd.items()

In [ ]:
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
trigramFreqTable.head().reset_index(drop=True)

,trigram,freq
0,"(aarogya, setu, app)",5588
1,"(download, aarogya, setu)",959
2,"(setu, app, not)",836
3,"(aarogya, setu, not)",716
4,"(use, aarogya, setu)",399


In [ ]:
trigramFreqTable[:10]

,trigram,freq
50,"(aarogya, setu, app)",5588
610,"(download, aarogya, setu)",959
67,"(setu, app, not)",836
12,"(aarogya, setu, not)",716
3414,"(use, aarogya, setu)",399
575,"(not, aarogya, setu)",364
3096,"(create, aarogya, setu)",274
466,"(self, declaration, form)",248
8159,"(setu, app, mandatory)",223
24631,"(setu, app, help)",207


In [ ]:
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filtered_tri[:10]

,trigram,freq
50,"(aarogya, setu, app)",5588
610,"(download, aarogya, setu)",959
3414,"(use, aarogya, setu)",399
466,"(self, declaration, form)",248
8159,"(setu, app, mandatory)",223
24631,"(setu, app, help)",207
1080,"(install, aarogya, setu)",204
1761,"(contact, trace, app)",184
41401,"(breach, aarogya, setu)",181
1406,"(setu, app, httpst)",172


In [ ]:
freq_bi = filtered_bi[:20].bigram.values

In [ ]:
freq_tri = filtered_tri[:20].trigram.values

## 2. PMI

In [ ]:
bigramFinder.apply_freq_filter(20)

In [ ]:
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)

In [ ]:
bigramPMITable[:10]

,bigram,PMI
0,"(cyberbullye, cyberthreat)",13.573951
1,"(cyberattack, cyberbullye)",13.442707
2,"(cybercrime, cyberattack)",13.375593
3,"(tik, tok)",13.322413
4,"(chairman, empower)",13.243803
5,"(powerful, companion)",12.756815
6,"(niti, aayog)",12.689701
7,"(datasecurity, dataprivacy)",12.644341
8,"(architecture, indulge)",12.637798
9,"(dc, gurugram)",12.566862


In [ ]:
trigramFinder.apply_freq_filter(20)

In [ ]:
trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)

In [ ]:
trigramPMITable[:10]

,trigram,PMI
0,"(cyberattack, cyberbullye, cyberthreat)",27.016658
1,"(cybercrime, cyberattack, cyberbullye)",26.949544
2,"(chairman, empower, group)",25.255875
3,"(cybersecurity, cybercrime, cyberattack)",25.254399
4,"(dataprivacy, cybersecurity, cybercrime)",24.456033
5,"(datasecurity, dataprivacy, cybersecurity)",24.452757
6,"(ravi, shankar, prasad)",24.423183
7,"(powerful, companion, protect)",23.356762
8,"(hardeep, singh, puri)",22.759892
9,"(data, datasecurity, dataprivacy)",22.676034


In [ ]:
pmi_bi = bigramPMITable[:20].bigram.values

In [ ]:
pmi_tri = trigramPMITable[:20].trigram.values

## 3. t-test

In [ ]:
bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)

In [ ]:
bigramTtable.head()

,bigram,t
0,"(aarogya, setu)",96.265038
1,"(setu, app)",70.045601
2,"(aarogyasetu, app)",28.682156
3,"(download, aarogya)",28.316578
4,"(app, not)",24.922853


In [ ]:
filteredT_bi = bigramTtable[bigramTtable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredT_bi[:10]

,bigram,t
0,"(aarogya, setu)",96.265038
1,"(setu, app)",70.045601
2,"(aarogyasetu, app)",28.682156
3,"(download, aarogya)",28.316578
6,"(contact, trace)",17.999391
8,"(self, declaration)",17.224211
9,"(smart, phone)",16.981410
10,"(declaration, form)",16.942790
12,"(use, aarogya)",15.795171
15,"(create, aarogya)",15.273181


In [ ]:
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','t']).sort_values(by='t', ascending=False)

In [ ]:
trigramTtable.head()

,trigram,t
0,"(aarogya, setu, app)",74.540741
1,"(download, aarogya, setu)",30.861601
2,"(setu, app, not)",28.295935
3,"(aarogya, setu, not)",26.070304
4,"(use, aarogya, setu)",19.806456


In [ ]:
filteredT_tri = trigramTtable[trigramTtable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filteredT_tri.head(10)

,trigram,t
0,"(aarogya, setu, app)",74.540741
1,"(download, aarogya, setu)",30.861601
4,"(use, aarogya, setu)",19.806456
7,"(self, declaration, form)",15.747960
8,"(setu, app, mandatory)",14.806361
9,"(setu, app, help)",14.279039
10,"(install, aarogya, setu)",14.227459
11,"(contact, trace, app)",13.561768
13,"(breach, aarogya, setu)",13.412753
15,"(immunity, download, aarogya)",13.070519


In [ ]:
t_bi = filteredT_bi[:20].bigram.values

In [ ]:
t_tri = filteredT_tri[:20].trigram.values

## 4. Chi-Square

In [ ]:
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [ ]:
bigramChiTable.head(20)

,bigram,chi-sq
0,"(tik, tok)",256086.000000
1,"(aarogya, setu)",247330.347503
2,"(cyberbullye, cyberthreat)",243890.476116
3,"(laxman, rekha)",243788.157344
4,"(cybercrime, cyberattack)",223186.796208
5,"(cyberattack, cyberbullye)",222680.869361
6,"(chairman, empower)",193999.393514
7,"(powerful, companion)",186866.269501
8,"(adapt, saptapati)",181530.798425
9,"(dc, gurugram)",175944.609899


In [ ]:
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)

In [ ]:
trigramChiTable.head(20)

,trigram,chi-sq
0,"(cyberattack, cyberbullye, cyberthreat)",2.715530e+09
1,"(cybercrime, cyberattack, cyberbullye)",2.592097e+09
2,"(cybersecurity, cybercrime, cyberattack)",8.405256e+08
3,"(chairman, empower, group)",8.013209e+08
4,"(ravi, shankar, prasad)",7.423783e+08
5,"(dataprivacy, cybersecurity, cybercrime)",4.833024e+08
6,"(datasecurity, dataprivacy, cybersecurity)",4.592447e+08
7,"(hardeep, singh, puri)",4.474564e+08
8,"(powerful, companion, protect)",2.900346e+08
9,"(within, laxman, rekha)",2.268846e+08


In [ ]:
chi_bi = bigramChiTable[:20].bigram.values

In [ ]:
chi_tri = trigramChiTable[:20].trigram.values

## 5. Likelihood

In [ ]:
bigramLikTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.likelihood_ratio)), columns=['bigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [ ]:
bigramLikTable.head()

,bigram,likelihood ratio
0,"(aarogya, setu)",81453.592967
1,"(setu, app)",26958.375274
2,"(mohfw, india)",3924.261659
3,"(declaration, form)",3852.122694
4,"(contact, trace)",3682.483445


In [ ]:
filteredLik_bi = bigramLikTable[bigramLikTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filteredLik_bi.head(10)

,bigram,likelihood ratio
0,"(aarogya, setu)",81453.592967
1,"(setu, app)",26958.375274
3,"(declaration, form)",3852.122694
4,"(contact, trace)",3682.483445
5,"(self, declaration)",3674.420570
6,"(download, aarogya)",3506.765970
7,"(aarogyasetu, app)",3236.842788
8,"(smart, phone)",3075.159953
9,"(social, distancing)",2964.571205
10,"(employee, respect)",2406.115714


In [ ]:
trigramLikTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.likelihood_ratio)), columns=['trigram','likelihood ratio']).sort_values(by='likelihood ratio', ascending=False)

In [ ]:
trigramLikTable.head()

,trigram,likelihood ratio
0,"(aarogya, setu, app)",162652.821953
1,"(download, aarogya, setu)",127521.141500
2,"(create, aarogya, setu)",123798.910611
3,"(breach, aarogya, setu)",123248.319367
4,"(register, aarogya, setu)",123246.714057


In [ ]:
filteredLik_tri = trigramLikTable[trigramLikTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
filteredLik_tri.head(20)

,trigram,likelihood ratio
0,"(aarogya, setu, app)",162652.821953
1,"(download, aarogya, setu)",127521.141500
3,"(breach, aarogya, setu)",123248.319367
4,"(register, aarogya, setu)",123246.714057
5,"(use, aarogya, setu)",123210.776837
6,"(setu, aarogya, setu)",123205.639788
7,"(install, aarogya, setu)",123205.621565
8,"(aarogya, setu, aap)",122894.353022
9,"(aarogya, setu, rti)",122791.442335
11,"(instal, aarogya, setu)",122704.570410


In [ ]:
lik_bi = filteredLik_bi[:20].bigram.values

In [ ]:
lik_tri = filteredLik_tri[:20].trigram.values

## Bigram Comparison

In [ ]:
bigramsCompare = pd.DataFrame([freq_bi, pmi_bi, t_bi, chi_bi, lik_bi]).T

In [ ]:
bigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test', 'Likeihood Ratio Test With Filter']

In [ ]:
bigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test,Likeihood Ratio Test With Filter
0,"(aarogya, setu)","(cyberbullye, cyberthreat)","(aarogya, setu)","(tik, tok)","(aarogya, setu)"
1,"(setu, app)","(cyberattack, cyberbullye)","(setu, app)","(aarogya, setu)","(setu, app)"
2,"(aarogyasetu, app)","(cybercrime, cyberattack)","(aarogyasetu, app)","(cyberbullye, cyberthreat)","(declaration, form)"
3,"(download, aarogya)","(tik, tok)","(download, aarogya)","(laxman, rekha)","(contact, trace)"
4,"(use, aarogya)","(chairman, empower)","(contact, trace)","(cybercrime, cyberattack)","(self, declaration)"
5,"(app, httpst)","(powerful, companion)","(self, declaration)","(cyberattack, cyberbullye)","(download, aarogya)"
6,"(contact, trace)","(niti, aayog)","(smart, phone)","(chairman, empower)","(aarogyasetu, app)"
7,"(self, declaration)","(datasecurity, dataprivacy)","(declaration, form)","(powerful, companion)","(smart, phone)"
8,"(smart, phone)","(architecture, indulge)","(use, aarogya)","(adapt, saptapati)","(social, distancing)"
9,"(declaration, form)","(dc, gurugram)","(create, aarogya)","(dc, gurugram)","(employee, respect)"


## Trigram Comparison

In [ ]:
trigramsCompare = pd.DataFrame([freq_tri, pmi_tri, t_tri, chi_tri, lik_tri]).T

In [ ]:
trigramsCompare.columns = ['Frequency With Filter', 'PMI', 'T-test With Filter', 'Chi-Sq Test', 'Likeihood Ratio Test With Filter']

In [ ]:
trigramsCompare

,Frequency With Filter,PMI,T-test With Filter,Chi-Sq Test,Likeihood Ratio Test With Filter
0,"(aarogya, setu, app)","(cyberattack, cyberbullye, cyberthreat)","(aarogya, setu, app)","(cyberattack, cyberbullye, cyberthreat)","(aarogya, setu, app)"
1,"(download, aarogya, setu)","(cybercrime, cyberattack, cyberbullye)","(download, aarogya, setu)","(cybercrime, cyberattack, cyberbullye)","(download, aarogya, setu)"
2,"(use, aarogya, setu)","(chairman, empower, group)","(use, aarogya, setu)","(cybersecurity, cybercrime, cyberattack)","(breach, aarogya, setu)"
3,"(self, declaration, form)","(cybersecurity, cybercrime, cyberattack)","(self, declaration, form)","(chairman, empower, group)","(register, aarogya, setu)"
4,"(setu, app, mandatory)","(dataprivacy, cybersecurity, cybercrime)","(setu, app, mandatory)","(ravi, shankar, prasad)","(use, aarogya, setu)"
5,"(setu, app, help)","(datasecurity, dataprivacy, cybersecurity)","(setu, app, help)","(dataprivacy, cybersecurity, cybercrime)","(setu, aarogya, setu)"
6,"(install, aarogya, setu)","(ravi, shankar, prasad)","(install, aarogya, setu)","(datasecurity, dataprivacy, cybersecurity)","(install, aarogya, setu)"
7,"(contact, trace, app)","(powerful, companion, protect)","(contact, trace, app)","(hardeep, singh, puri)","(aarogya, setu, aap)"
8,"(breach, aarogya, setu)","(hardeep, singh, puri)","(breach, aarogya, setu)","(powerful, companion, protect)","(aarogya, setu, rti)"
9,"(setu, app, httpst)","(data, datasecurity, dataprivacy)","(immunity, download, aarogya)","(within, laxman, rekha)","(instal, aarogya, setu)"
